# Neuronale Netze

## Quelle der Daten

https://www.kaggle.com/datasets/captainozlem/framingham-chd-preprocessed-data (zuletzt aufgerufen: 01/2024)

https://www.framinghamheartstudy.org/ (zuletzt aufgerufen: 01/2024)

## Installation der Bibliotheken

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

import tensorflow as tf

## Einlesen der Daten

In [ ]:
data_url = "https://github.com/timwgnd/Lehrbuch-Kuenstliche-Intelligenz-in-der-Medizin/raw/refs/heads/main/FraminghamHeartStudy.xlsx"
data = pd.read_excel(io=data_url, sheet_name = "Tabelle1")

data = data.dropna()

data.head()

In [ ]:
data_new = pd.get_dummies(data, columns = ["Geschlecht"])

data_new.head()

In [ ]:
count_diabetes = data_new["Diabetes"].value_counts()

count_diabetes.plot(kind = "bar", rot = 0)

## Aufteilung der Daten und Resampling

In [ ]:
x = data_new.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]]

y = data_new.iloc[:, 7]

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

In [ ]:
sm = SMOTE()

x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

print(y_train_res.value_counts())

## FCNN-Architektur

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid))

## Modelltraining und -evaluation

In [ ]:
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = "accuracy")

In [ ]:
"""
tf.keras.optimizers.Adam(
    learning_rate = 0.001, 
    beta_1 = 0.9, 
    beta_2 = 0.999, 
    epsilon = 1e-07, 
    amsgrad = False, 
    name = "adam")

tf.keras.losses.BinaryCrossentropy(
    from_logits = False,
    label_smoothing = 0.0,
    axis = -1,
    name = "binary_crossentropy")

tf.keras.metrics.Accuracy(
    name = "accuracy")
"""

In [ ]:
model.fit(x_train_res, y_train_res, epochs = 10)

In [ ]:
model.evaluate(x_test, y_test)